# Lab 2 Part 2: Dory's Cost Surface

In [5]:
import arcpy
import requests
import zipfile
import os
import numpy as np

In [14]:
#created a CSV with dory's start and end points, add to map
arcpy.management.XYTableToPoint(
    in_table = "C:/Users/schil726/Downloads/dory_coords.csv",
    out_feature_class = "dory_AOI",
    x_field = "x",
    y_field = "y",
    coordinate_system = arcpy.SpatialReference(4326),
)

<Result 'C:\\Users\\schil726\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2Pt2\\GIS5571_Lab2Pt2.gdb\\dory_AOI'>

In [15]:
#buffer dorys points, this will serve as mask for data inputs
arcpy.analysis.Buffer(
    in_features = "dory_AOI",
    out_feature_class = "dory_buffered_AOI",
    buffer_distance_or_field = "5 Miles",
    line_side = "FULL",
    line_end_type = "ROUND",
    dissolve_option = "ALL",
)

<Result 'C:\\Users\\schil726\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2Pt2\\GIS5571_Lab2Pt2.gdb\\dory_buffered_AOI'>

In [25]:
#define urls and make calls (in one big block) for data we need

#30m DEM
DEM_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip"
DEM_response = requests.get(DEM_url)

#land cover
landcover_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip"
landcover_response = requests.get(landcover_url)

#road centerlines
roads_url ="https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_roads_centerlines/shp_trans_roads_centerlines.zip"
roads_response = requests.get(roads_url)

#bridges
bridges_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_bridges/shp_trans_bridges.zip"
bridges_response = requests.get(bridges_url)

#streams
streams_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/water_strahler_stream_order/shp_water_strahler_stream_order.zip'
streams_response = requests.get(streams_url)

In [26]:
#i have too many urls so lets define a function to save and extract
def save_locally_extract(response_dict, save_path):
    for file_name, response in response_dict.items():
         
        with open(file_name, "wb") as file:
            file.write(response.content)
        
        with zipfile.ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall(save_path)

responses = {
    "DEM.zip": DEM_response,
    "landcover.zip": landcover_response,
    "roads.zip": roads_response,
    "bridges.zip": bridges_response,
    "streams.zip": streams_response
}
   
savepath = "C:/Users/schil726/Downloads/GIS5571_Lab2/part2"

save_locally_extract(responses, savepath)

In [54]:
#need all raster layers for cost surface
def convert_to_raster(layers, cell_size, output_path):
    for layer_name in layers:
        output_raster = f"{output_path}/{layer_name}_raster.tif"
        
        desc = arcpy.Describe(layer_name)
        if desc.shapeType == "Point":
            arcpy.conversion.PointToRaster(
                in_features = layer_name,
                value_field = "FID",
                out_rasterdataset = output_raster,
                cell_assignment = "MAXIMUM",
                cellsize = cell_size
            )
            
        elif desc.shapeType == "Polyline":
            arcpy.conversion.PolylineToRaster(
                in_features = layer_name,
                value_field = "FID",
                out_rasterdataset = output_raster,
                cellsize = cell_size
            )
            
vector_layers = ["streams", "roads", "bridges"]
cell_size = 30
output_dir = "C:/Users/schil726/Downloads/GIS5571_Lab2/part2/rasters"
        
convert_to_raster(vector_layers, cell_size, output_dir)

In [56]:
#create slope from DEM
arcpy.ddd.Slope(
    in_raster = "30m_DEM",
    out_raster = "slope",
)

<Result 'C:\\Users\\schil726\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2Pt2\\GIS5571_Lab2Pt2.gdb\\slope'>

In [61]:
#clip by the buffered AOI (extract by mask)
def clip_by_mask(raster_layers, mask, output_path):
        for raster in raster_layers:
            out_extract = arcpy.sa.ExtractByMask(
                in_raster = raster,
                in_mask_data = mask,
                extraction_area = "INSIDE"
                )
            out_extract.save(f"{output_dir}/{raster}_clipped.tif")
        
raster_layers = ["roads_raster.tif", "streams_raster.tif", "bridges_raster.tif", "NLCD_landcover", "30m_DEM", "slope"]
mask = "dory_buffered_AOI"
output_dir = "C:/Users\schil726/Downloads/GIS5571_Lab2/part2/clipped"

clip_by_mask(raster_layers, mask, output_dir)

## reclass time!

In [67]:
#reclassify NLCD
arcpy.ddd.Reclassify(
    in_raster = "NLCD_landcover_clipped.tif",
    reclass_field = "NLCD_Land", 
    remap = "'Open Water' 10;'Developed, Open Space' 2;'Developed, Low Intensity' 2;'Developed, Medium Intensity' 2;'Developed, High Intensity' 2;'Barren Land' 5;'Deciduous Forest' 7;'Evergreen Forest' 7;'Mixed Forest' 7;Shrub/Scrub 7;Herbaceous 7;Hay/Pasture 9;'Cultivated Crops' 9;'Woody Wetlands' 10;'Emergent Herbaceous Wetlands' 10", 
    out_raster = "C:/Users/schil726/Downloads/GIS5571_Lab2/part2/reclass/NLCD_landcover_reclass.tif",
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_Lab2\\part2\\reclass\\NLCD_landcover_reclass.tif'>

In [68]:
#reclassify slope
arcpy.ddd.Reclassify(
    in_raster = "slope_clipped.tif",
    reclass_field = "VALUE",
    remap = "0 2.940712 1;2.940712 6.763637 2;6.763637 11.762846 3;11.762846 21.173123 4;21.173123 74.988144 5",
    out_raster = "C:/Users/schil726/Downloads/GIS5571_Lab2/part2/reclass/slope_reclass.tif",
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_Lab2\\part2\\reclass\\slope_reclass.tif'>

In [69]:
#reclassify streams
arcpy.ddd.Reclassify(
   in_raster = "streams_raster.tif_clipped.tif",
    reclass_field = "VALUE",
    remap = "1 1;2 2;3 3;4 7;5 8;6 9;8 10",
    out_raster = "C:/Users/schil726/Downloads/GIS5571_Lab2/part2/reclass/stream_reclass.tif",
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_Lab2\\part2\\reclass\\stream_reclass.tif'>

In [70]:
#reclassify bridges
arcpy.ddd.Reclassify(
   in_raster = "bridges_raster.tif_clipped.tif",
    reclass_field = "VALUE",
    remap = "1 1;2 2;3 3;4 7;5 8;6 9;8 10",
    out_raster = "C:/Users/schil726/Downloads/GIS5571_Lab2/part2/reclass/bridge_reclass.tif",
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_Lab2\\part2\\reclass\\bridge_reclass.tif'>

In [71]:
arcpy.ddd.Reclassify(
   in_raster = "roads_raster.tif_clipped.tif",
    reclass_field = "VALUE",
    remap = "1 1;2 2;3 3;4 7;5 8;6 9;8 10",
    out_raster = "C:/Users/schil726/Downloads/GIS5571_Lab2/part2/reclass/road_reclass.tif",
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_Lab2\\part2\\reclass\\road_reclass.tif'>

#### was there a better way to do that (i.e. define a function)? yes. but it's ok.

## cost surface

In [13]:
#weight my layers
weight_slope = 0.5
weight_landcover = 0.2
weight_streams = 0.1
weight_roads = 0.1
weight_bridges = 0.1

#define inputs
slope_raster = "slope_reclass.tif"
landcover_raster = "NLCD_landcover_reclass.tif"
streams_raster = "stream_reclass.tif"
bridges_raster = "bridge_reclass.tif"
roads_raster = "road_reclass.tif"

#create cost surface raster
weighted_cost_surface = (
    (arcpy.sa.Raster(slope_raster)*weight_slope) +
    (arcpy.sa.Raster(landcover_raster)*weight_landcover) +
    (arcpy.sa.Raster(streams_raster)*weight_streams )+
    (arcpy.sa.Raster(roads_raster)*weight_roads) +
    (arcpy.sa.Raster(bridges_raster)*weight_bridges)
)

output_cost_surface_raster = "C:/Users/schil726/Downloads/GIS5571_Lab2/part2/costsurface/cost_surface_raster.tif"
weighted_cost_surface.save(output_cost_surface_raster)

In [15]:
#create cost distance
out_cost_distance = (arcpy.sa.CostDistance(
    in_source_data = "dory_AOI", 
    in_cost_raster = "weighted_cost_surface"
))

out_cost_distance.save("C:/Users/schil726/Downloads/GIS5571_Lab2/part2/costsurface/cost_distance.tif")